In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD
import joblib

In [7]:
md = pd.read_csv('./sources/best_metadata_big.csv')
md.head()

,tmdbId,title,description,year,original_title,poster_path,release_date,vote_average,vote_count,soup
0,88224,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,Toy Story,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,7.7,5415,jealousi toy boy friendship friend rivalri boy...
1,42164,Jumanji,When siblings Judy and Peter discover an encha...,1995,Jumanji,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,6.9,2413,disappear basedonchildren'sbook newhom reclus ...
2,220,Heat,"Obsessive master thief, Neil McCauley leads a ...",1995,Heat,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,7.7,1886,robberi detect bank obsess chase shoot thief h...
3,23449,GoldenEye,James Bond must unmask the mysterious head of ...,1995,GoldenEye,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,1995-11-16,6.6,1194,cuba falselyaccus secretident computervirus se...
4,9361,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",1995,Cutthroat Island,/odM9973kIv9hcjfHPp6g6BlyTIJ.jpg,1995-12-22,5.7,137,exoticisland treasur map ship pirat geenadavis...


In [8]:
# create count vectorizer
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(md['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(md.index, index=md['tmdbId'])

# disable copy warning when calling improved_recommendations
pd.set_option('mode.chained_assignment', None)

In [9]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
def improved_recommendations(tmdbId):
    index = indices[tmdbId]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = md.iloc[movie_indices][['tmdbId', 'title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    m = vote_counts.quantile(0.60)
    C = vote_averages.mean()
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified.vote_count = qualified.vote_count.astype('int')
    qualified.vote_count = qualified.vote_count.astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [11]:
# tmdbId for 'The Dark Knight'
improved_recommendations(1873)

NameError: name 'm' is not defined

In [12]:
# save pre-processing objects and algorithm
joblib.dump(md, "./metadata.joblib", compress=True)
joblib.dump(cosine_sim, "./cosine_sim.joblib", compress=True)

['./cosine_sim.joblib']